In [1]:
import os
import pandas as pd
from sklearn.metrics import f1_score
import ast
import numpy as np


## Utils

In [2]:
def get_acc1_f1(df):
    acc1 = (df['prediction'] == df['ground_truth']).sum() / len(df)
    f1 = f1_score(df['ground_truth'], df['prediction'], average='weighted')
    return acc1, f1

def get_is_correct(row):
    pred_list = row['prediction']
    if row['ground_truth'] in pred_list:
        row['is_correct'] = True
    else:
        row['is_correct'] = False
    
    return row


def get_is_correct10(row):
    pred_list = row['top10']
    if row['ground_truth'] in pred_list:
        row['is_correct10'] = True
    else:
        row['is_correct10'] = False
        
    pred_list = row['top5']
    if row['ground_truth'] in pred_list:
        row['is_correct5'] = True
    else:
        row['is_correct5'] = False

    pred = row['top1']
    if pred == row['ground_truth']:
        row['is_correct1'] = True
    else:
        row['is_correct1'] = False
    
    return row


def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)


def get_ndcg(prediction, targets, k=10):
    """
    Calculates the NDCG score for the given predictions and targets.

    Args:
        prediction (Nxk): list of lists. the softmax output of the model.
        targets (N): torch.LongTensor. actual target place id.

    Returns:
        the sum ndcg score
    """
    for _, xi in enumerate(prediction):
        if len(xi) < k:
            #print(f"the {i}th length: {len(xi)}")
            xi += [-5 for _ in range(k-len(xi))]
        elif len(xi) > k:
            xi = xi[:k]
        else:
            pass
    
    n_sample = len(prediction)
    prediction = np.array(prediction)
    targets = np.broadcast_to(targets.reshape(-1, 1), prediction.shape)
    hits = first_nonzero(prediction == targets, axis=1, invalid_val=-1)
    hits = hits[hits>=0]
    ranks = hits + 1
    ndcg = 1 / np.log2(ranks + 1)
    return np.sum(ndcg) / n_sample
    

## Top10

In [7]:
# Calculate the metric for all user
output_dir = 'output/fsq/top10'
file_list = [file for file in os.listdir(output_dir) if file.endswith('.csv')]
print(file_list)
print(f"[DEBUG] CSV文件总数: {len(file_list)}")
file_path_list = [os.path.join(output_dir, file) for file in file_list]
# file_path_list = file_path_list[:1]
# print(file_path_list)

df = pd.DataFrame({
    'user_id': None,
    'ground_truth': None,
    'prediction': None,
    'reason': None
}, index=[])

for file_path in file_path_list:
    iter_df = pd.read_csv(file_path)
    if output_dir[-1] != '1':
        pred_series = iter_df['prediction'].apply(lambda x: ast.literal_eval(x))  # A pandas series
        iter_df['top10'] = pred_series.apply(lambda x: x[:10] if type(x) == list else [x] * 10)
        iter_df['top5'] = pred_series.apply(lambda x: x[:5] if type(x) == list else [x] * 5)
        iter_df['top1'] = pred_series.apply(lambda x: x[0] if type(x) == list else x)
        # iter_df['prediction'] = iter_df['prediction'].apply(lambda x: x.strip('][').split(', '))
    df = pd.concat([df, iter_df], ignore_index=True)
    

df = df.apply(func=get_is_correct10, axis=1)

print("number of samples: ", len(df))

acc1 = (df['is_correct1']).sum() / len(df)
acc5 = (df['is_correct5']).sum() / len(df)
acc10 = (df['is_correct10']).sum() / len(df)
f1 = f1_score(df['ground_truth'], df['top1'], average='weighted')
preds = df['top10'].tolist()
targets = np.array(df['ground_truth'].tolist())
ndcg = get_ndcg(prediction=preds, targets=targets, k=10)

print("Acc@1: ", acc1)
print("Acc@5: ", acc5)
print("Acc@10: ", acc10)
print("Weighted F1: ", f1)
print("NDCG@10: ", ndcg)

['01.csv', '02.csv', '03.csv', '04.csv', '05.csv', '06.csv', '07.csv', '08.csv', '09.csv', '10.csv', '100.csv', '101.csv', '102.csv', '103.csv', '104.csv', '105.csv', '106.csv', '107.csv', '108.csv', '109.csv', '11.csv', '110.csv', '111.csv', '112.csv', '113.csv', '114.csv', '115.csv', '116.csv', '117.csv', '118.csv', '119.csv', '12.csv', '120.csv', '121.csv', '122.csv', '123.csv', '124.csv', '125.csv', '126.csv', '127.csv', '128.csv', '129.csv', '13.csv', '130.csv', '131.csv', '132.csv', '133.csv', '134.csv', '135.csv', '136.csv', '137.csv', '138.csv', '139.csv', '14.csv', '140.csv', '141.csv', '142.csv', '143.csv', '144.csv', '145.csv', '146.csv', '147.csv', '148.csv', '149.csv', '15.csv', '150.csv', '151.csv', '152.csv', '153.csv', '154.csv', '155.csv', '156.csv', '157.csv', '158.csv', '159.csv', '16.csv', '160.csv', '161.csv', '162.csv', '163.csv', '164.csv', '165.csv', '166.csv', '167.csv', '168.csv', '169.csv', '17.csv', '170.csv', '171.csv', '172.csv', '173.csv', '174.csv', '175

# Top1

In [13]:
# Calculate the metric for all user
output_dir = 'output/fsq/top1_wot'
file_list = [file for file in os.listdir(output_dir) if file.endswith('.csv')]
print(file_list)
file_path_list = [os.path.join(output_dir, file) for file in file_list]

df = pd.DataFrame({
    'user_id': None,
    'ground_truth': None,
    'prediction': None,
    'reason': None
}, index=[])

for file_path in file_path_list:
    iter_df = pd.read_csv(file_path)
    df = pd.concat([df, iter_df], ignore_index=True)

df['prediction'] = df['prediction'].apply(lambda x: int(x))
df['ground_truth'] = df['ground_truth'].apply(lambda x: int(x))

acc1, f1 = get_acc1_f1(df)
print("Acc@1: ", acc1)
print("F1: ", f1)

['01.csv', '02.csv', '03.csv', '04.csv', '05.csv', '06.csv', '07.csv', '08.csv', '09.csv', '10.csv', '100.csv', '101.csv', '102.csv', '103.csv', '104.csv', '105.csv', '106.csv', '107.csv', '108.csv', '109.csv', '11.csv', '110.csv', '111.csv', '112.csv', '113.csv', '114.csv', '115.csv', '116.csv', '117.csv', '118.csv', '119.csv', '12.csv', '120.csv', '121.csv', '122.csv', '123.csv', '124.csv', '125.csv', '126.csv', '127.csv', '128.csv', '129.csv', '13.csv', '130.csv', '131.csv', '132.csv', '133.csv', '134.csv', '135.csv', '136.csv', '137.csv', '138.csv', '139.csv', '14.csv', '140.csv', '141.csv', '142.csv', '143.csv', '144.csv', '145.csv', '146.csv', '147.csv', '148.csv', '149.csv', '15.csv', '150.csv', '151.csv', '152.csv', '153.csv', '154.csv', '155.csv', '156.csv', '157.csv', '158.csv', '159.csv', '16.csv', '160.csv', '161.csv', '162.csv', '163.csv', '164.csv', '165.csv', '166.csv', '167.csv', '168.csv', '169.csv', '17.csv', '170.csv', '171.csv', '172.csv', '173.csv', '174.csv', '175